In [20]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import namedtuple
from dgl.nn.pytorch import GATConv
from torch.optim import Adam

## Observation Encoder

\begin{equation}
h_{i}=\mathrm{MLP}(o_{i})
\end{equation}

In [2]:
class ObsEncoder(nn.Module):
    def __init__(self, in_dim, o_dim=128, h_dim=512):
        super(ObsEncoder, self).__init__()
        self.fc1 = nn.Linear(in_dim. h_dim)
        self.fc2 = nn.Linear(h_dim, o_dim)
    
    def forward(self, o):
        o = F.relu(self.fc1(o))
        o = F.relu(self.fc2(o))
        return o

## Relational Kernel

\begin{equation}
\alpha_{i,j}^{m}=\frac{\exp(\tau\cdot \mathbf{W}_{Q}^{m}h_{i}\cdot(\mathbf{W}_{K}^{m}h_{j})^\top)}{\sum_{k\in\mathbb{B}_{+i}}\exp(\tau\cdot\mathbf{W}_{Q}^{m}h_{i}\cdot(\mathbf{W}_{K}^{m}h_{k})^{\top})}
\end{equation}

\begin{equation}
h_{i}^{'}=\sigma\left( \mathrm{concat}_{m\in M}\left[ \sum_{j\in\mathbb{B}_{+i}}\alpha_{i,j}^{m}\mathbf{W}_{v}^{m}h_{j} \right] \right)
\end{equation}



In [13]:
class DotGATLayer(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(DotGATLayer, self).__init__()
        self.fc_q = nn.Linear(in_dim, out_dim)
        self.fc_k = nn.Linear(in_dim, out_dim)
        self.fc_v = nn.Linear(in_dim, out_dim)
        self.tau = 1/math.sqrt(out_dim)

    def edge_attention(self, edges):
        k = self.fc_k(edges.src['z'])
        q = self.fc_q(edges.dst['z'])
        a = (k*q).sum(-1, keepdims=True)*self.tau
        return {'e': a}

    def message_func(self, edges):
        return {'z': edges.src['z'], 'e': edges.data['e']}
    
    def reduce_func(self, nodes):
        s = nodes.mailbox['e']
        alpha = F.softmax(s, dim=1)
        v = self.fc_v(nodes.mailbox['z'])
        h = torch.sum(alpha * v, dim=1)
        return {'h': h, 'alpha': alpha.squeeze()}

    def forward(self, g, z):
        g.ndata['z'] = z
        g.apply_edges(self.edge_attention)
        g.update_all(self.message_func, self.reduce_func)
        h = g.ndata.pop('h')
        alpha = g.ndata.pop('alpha')
        return h, alpha

class MultiHeadDotGATLayer(nn.Module):
    def __init__(self, in_dim, out_dim,
                 num_heads, merge='cat'):
        super(MultiHeadDotGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        h_dim = out_dim // num_heads
        assert (h_dim*num_heads) == out_dim
        for _ in range(num_heads):
            self.heads.append(DotGATLayer(in_dim, h_dim))
        self.merge = merge

    def forward(self, g, h):
        hs, alphas = map(list, zip(*[head(g, h)
                                     for head in self.heads]))
        alpha = torch.stack(alphas).mean(0)
        if self.merge == 'cat':
            h = torch.cat(hs, dim=1)
            return h, alpha
        else:
            return torch.mean(torch.stack(hs)), alpha

## DGN-R Agent

\begin{equation}
Q(o_{i}, \cdot)=\mathrm{Linear}\left(\mathrm{concat}\left[ h_{i}, h_{i}^{'}, h_{i}^{''} \right]\right)
\end{equation}

\begin{equation}
\mathcal{L}_{\mathrm{reg}}(\theta)=\frac{1}{M}\sum_{m=1}^{M}D_{\mathrm{KL}}\left( \mathcal{G}_{m}(O_{i,\mathcal{C}};\theta) || \mathcal{G}_{m}(O_{i,\mathcal{C}}';\theta) \right)
\end{equation}

In [19]:
Transition = namedtuple('Transition', ('graph', 'action', 'reward', 'next_graph', 'done'))


class ReplayBuffer(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Save Transitions"""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        samples = random.sample(self.memory, batch_size)
        
        graphs = [sample[0] for sample in samples]
        actions = [sample[1] for sample in samples]
        rewards = [sample[2] for sample in samples]
        next_graphs = [sample[3] for sample in samples]
        dones = [sample[4] for sample in samples]

        ret_graph = dgl.batch(graphs)
        ret_action = torch.stack(actions).reshape(-1, 1)
        ret_reward = torch.Tensor(rewards).reshape(-1)
        ret_next_graph = dgl.batch(next_graphs)
        ret_dones = torch.Tensor(dones).reshape(-1)

        
        return ret_graph, ret_action, ret_reward, ret_next_graph, ret_dones

    def __len__(self):
        return len(self.memory)

In [22]:
class DGN_Conv(nn.Module):
    def __init__(self, obs_dim, h_dim=128, num_heads=8,
                 target=False):
        super(DGN_Q_Net, self).__init__()
        self.encoder = ObsEncoder(in_dim=obs_dim, o_dim=h_dim)
        self.conv1 = MultiHeadDotGATLayer(h_dim, h_dim, num_heads)
        self.conv2 = MultiHeadDotGATLayer(h_dim, h_dim, num_heads)
        self.target = target
    
    def forward(self, graph):
        obs = graph.ndata['obs']
        z1 = self.encoder(obs)
        z2, _ = self.conv1(z1)
        z3, alpha = self.conv2(z2)
        out = torch.cat([z1, z2, z3], dim=1)
        if self.target:
            return out
        return out, alpha

class DGNAgent(nn.Module):
    def __init__(self, n_agents, obs_dim, act_dim, h_dim=128,
                 num_heads=8, gamma=0.96, batch_size=10,
                 buffer_size=2*1e5, epsilon=0.6, episilon_min=0.01,
                 decay_rate=0.996, lr=1e-4, neighbors=3, tau=0.25,
                 lamb=0.03, beta=0.01, *args, **kwargs):
        super(DGNAgent, self).__init__()
        self.conv_net = DGN_Conv(obs_dim, h_dim, num_heads)
        self.target_conv = DGN_Conv(obs_dim, h_dim, num_heads, target=True)
        self.q_net = nn.Linear(3*h_dim, act_dim)
        self.target_q = nn.Linear(3*h_dim, act_dim)
        self.target_conv.load_state_dict(self.conv_net.state_dict())
        self.target_q.load_state_dict(self.q_net.state_dict())
        self.optimizer = Adam(self.conv_net.parameters()+self.q_net.parameters(),
                              lr=lr)
        self.beta = beta
        self.gamma = gamma
        self.buffer = ReplayBuffer(buffer_size)
        self.batch_size = batch_size

        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.decay_rate = decay_rate
        
        self.n_agents = n_agents
        self.n_act = act_dim

    def get_action(self, graph):
        if random.random() < self.epsilon:
            action = torch.randint(0, n_act, size=(self.n_agents,))
        else:
            q_value = self.q_net(graph)
            action = q.argmax(dim=-1).detach()
        self.epsilon = max(self.epsilon*self.decay_rate, self.epsilon_min)

        return action

    def get_q(self, graph):
        z, weight = self.conv_net(graph)
        q = self.q_net(z)
        return q, weight
    
    def get_target(self, graph):
        z = self.target_conv(graph)
        q = self.target_q(z)
        return q
    
    def save_samples(self, g, a, r, n_g, t):
        self.buffer.push(g, a, r, n_g, t)
    
    def fit(self):
        if len(self.buffer) < self.batch_size*10:
            return False, 0
        
        state, act, reward, n_state, done = self.buffer.sample(self.batch_size)
        curr_qs, curr_weight = self.get_qs(state)
        selected_qs = curr_qs.gather(1, act).reshape(-1)
        next_qs = self.get_target(n_state).max(dim=1)[0].detach()
        target = reward + self.gamma * next_qs * (1 - done)
        
        _, next_weight = self.get_qs(n_state)
        KL = (curr_weight * torch.log(curr_weight/next_weight)).sum(-1)
        
        loss = F.mse_loss(curr_qs, target) + self.lamb*KL.mean()
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.target_update()
        
        return True, loss.item()
    
    def target_update(self):
        for target, param in zip(self.target_conv.parameters(),
                                  self.conv_net.parameters()):
            target.data = (1-self.beta)*target.data + self.beta*param.data

        for target, param in zip(self.q_net.parameters(),
                                  self.target_q.parameters()):
            target.data = (1-self.beta)*target.data + self.beta*param.data

## Test code

In [6]:
import dgl

feature_dim = 3
num_nodes = 5

def get_fully_connected_edges(num_agents, self_edge=True):
    from_idx = []
    to_idx = []
    for i in range(num_agents):
        for j in range(num_agents):
            if self_edge:
                from_idx.append(i)
                to_idx.append(j)
            else:
                if i!=j:
                    from_idx.append(i)
                    to_idx.append(j)
    return from_idx, to_idx

edge_from, edge_to = get_fully_connected_edges(num_nodes)

ex_graph = dgl.DGLGraph()
ex_graph.add_nodes(num_nodes)
ex_graph.add_edges(edge_from, edge_to)

c:\users\bq173\appdata\local\programs\python\python37\lib\site-packages\dgl\base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [16]:
node_feature = torch.randn(num_nodes, feature_dim)
attn = MultiHeadDotGATLayer(feature_dim, feature_dim, num_heads=1)
h, alpha = attn(ex_graph, node_feature)
print(alpha)

tensor([[0.1323, 0.2330, 0.1995, 0.2321, 0.2031],
        [0.1354, 0.2126, 0.2086, 0.2276, 0.2157],
        [0.1538, 0.2071, 0.2219, 0.2267, 0.1905],
        [0.1535, 0.2047, 0.2191, 0.2241, 0.1986],
        [0.2957, 0.1780, 0.2056, 0.1761, 0.1447]], grad_fn=<MeanBackward1>)
